In [1]:
import cv2
import pandas as pd
from imutils import paths
import numpy as np

In [14]:
def extract_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [15]:
images_train = sorted(list(paths.list_images('data/train')))

In [16]:
result = []
for image in images_train:
  result.append(extract_histogram(cv2.imread(image)))
result = np.array(result)
X = pd.DataFrame(result)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.242645,0.073122,0.000232,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.087363,...,0.00000,0.000000,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000
1,0.418818,0.126781,0.000085,0.000000,0.000000,0.0,0.0,0.0,0.005085,0.076441,...,0.00000,0.000000,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.000042,0.000000
2,0.356737,0.184168,0.020616,0.001619,0.000717,0.0,0.0,0.0,0.000041,0.027440,...,0.00002,0.000000,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.007726
3,0.001266,0.019995,0.011941,0.001583,0.000158,0.0,0.0,0.0,0.000000,0.002145,...,0.01595,0.002233,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.023354,0.497077
4,0.000000,0.000105,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000366,...,0.00000,0.000000,0.0,0.002719,0.7748,0.34338,0.038583,0.114442,0.126833,0.008469


In [17]:
Y = [0 if 'cat' in image else 1 for image in images_train]

In [18]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [19]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.25, random_state=19)

In [20]:
lsvc = LinearSVC(C=0.6, random_state=19)
print(lsvc)

LinearSVC(C=0.6, random_state=19)


In [21]:
lsvc.fit(xtrain, ytrain)

LinearSVC(C=0.6, random_state=19)

In [25]:
coeff = pd.DataFrame(lsvc.coef_)
print(round(coeff[10],2))
print(round(coeff[12],2))
print(round(coeff[317], 2))
     

0   -0.42
Name: 10, dtype: float64
0   -0.08
Name: 12, dtype: float64
0   -0.09
Name: 317, dtype: float64


In [23]:
ypred = lsvc.predict(xtest)

cm = confusion_matrix(ytest, ypred)
print(cm)

[[79 25]
 [76 70]]


In [12]:

cr = classification_report(ytest, ypred)
print(cr)

              precision    recall  f1-score   support

           0       0.51      0.76      0.61       104
           1       0.74      0.48      0.58       146

    accuracy                           0.60       250
   macro avg       0.62      0.62      0.60       250
weighted avg       0.64      0.60      0.59       250



In [31]:
print((0.62 + 0.56) / 2)

0.5900000000000001


In [35]:
result = extract_histogram(cv2.imread('data/test/cat.1002.jpg'))
X_test = pd.DataFrame(result).T

print(lsvc.predict(X_test))


[1]
